# Batch gradient descent

For linear regression we could analytically find a closed-form solution for the root of the gradient of the cost function, i.e. the minimum or maximum points of the cost function. But for most models we won't be able to find close-form solutions.

A common optimisation algorithm is the steepest descent algorithm, also known as **gradient descent**:

$$
    \underline w^{(t + 1)} \leftarrow \underline w^(t) - \eta\nabla_{\underline w} C
$$

where the superscripts denote time points, $C$ is the cost function - a function of the weights, but we didn't write $C(\underline w)$ to simplify notation.
$$
    \nabla_{\underline w} C = \sum_{n=1}^N \nabla_{\underline w} C^{(n)}
$$
is the vector of first-order partial derivatives, where N is the number of training examples.
For instance, if $|w|=2$, $\nabla_{\underline w} = \left[ \frac{\partial\sum_n C^{(n)}}{\partial w_0}, \frac{\partial \sum_n C^{(n)}}{\partial w_1} \right]^T$.

At a certain location $\underline w$, the vector $\nabla_{\underline w} C$ points to the steepest **ascent** of $C$ from that location. So, if $(\nabla_{\underline w})_i = -3$:
- the <u>minus</u> sign in $-3$ indicates the direction of the steepest <u>ascent</u> with regards to axis $w_i$ is that which <u>decreases</u> $w_i$;
- the value $3$ in $-3$ is the magnitude of the gradient, i.e. the steepness of the slope, i.e. difference in function values, between current point $w_i$ and some point $w_i - \epsilon$ before $w_i$. This is actually used to control the amount of change in gradient descent, alongside the learning rate, i.e. the change in $w_i$ is $-\eta (-3)$ here. It makes sense if we interpret the magnitude as a confidence score: if the steepness is high, we might as well make a bigger step - but this is assuming there aren't any sharp, narrow drops that we skip over with a big step.

In situations where $C=\sum_n C^{(n)}$, the algorithm is also known as <u>batch gradient descent</u>.

## Advantages of gradient descent
- Option when we are unable to find a closed form solution for the root of the gradient of the cost function, i.e. the minimum or maximum points of the cost function.
- It might be even preferred to closed form solutions. In closed form solutions, we still need numerical algorithms for e.g. matrix inversion or solving systems of equations, which come with their limitations.

## Limitations of gradient descent
- Batch version: slow convergence because it is expensive: $C^{(n)} = \sum_n C^{(n)}$. We need to visit the entire training set to compute the gradient vector, which could consist of millions of examples. Solutions include minibatch and stochastic versions, see below.
- Batch, minibatch, and stochastic versions: slow convergence. We **need** to use a small learning rate.
  - They do not always move in the optimal direction, i.e. towards global minimum, since it only considers local gradient information. But local gradients only towards steepest ascent from current point.
  - But this can result in oscillations.
  - So we need to use a small learning rate to avoid going too far in a wrong direction.
  - Solutions include, non-linear conjugate gradients, L-BFGS, Newton's method;
  - Also momentum and others, see below.

# Minibatch and stochastic gradient descent
One limintation of batch gradient descent is that it is expensive. Since $C^{(n)} = \sum_n C^{(n)}$, we need to visit the entire training set to compute the gradient vector, which could consist of millions of examples.

But, intuitively, it might be enough to get massive cost on a few example to figure out our classifier is not doing a great job and weights need to be updated.

We could estimate the cost by considering a <u>random</u> sample (i.e. a minibatch) of $|B|$ examples as:
$$
   C^{(n)} = \sum_{n=1}^N C^{(n)} \approx \sum_{b=1}^{|B|} C^{(b)}.
$$
Note the importance of having a <u>random</u> sample to avoid overfitting. In this case, overfitting would materialise as optimising the cost to fit examples from a certain biased sample $B$ of the trainig set. This is called <u>minibatch gradient descent</u>. When $|B|=1$, it is called <u>stochastic gradient descent</u>.

# Momentum

Gradient descent only considers local gradient information. This can result in slow convergence:
- Local gradients do not necessarily point towards global optima, only towards steepest ascent from current point.
- But this can result in oscillations, making convergence slow.
- We cannot fix this with a large learning rate. Due to oscillations, this would risk going too far in a wrong direction. We need to keep the learning rate small.

Let's introduce the idea of exponentially weighted averages. Say we have a noisy time series $x_t$.

<img src="../img/3-gradients-time-series.png" width="200" height="100">

Transform it as $v_t = \beta v_{t-1} + (1 - \beta) x_t$, where $v_0 = 0$. $v_t$ is the average over the previous approx. $1 / (1 - \beta)$ steps.
Below we have $v_t$ for $\beta = 0.9$ (approx. 10 steps previous steps averaged; shown in red), $\beta = 0.98$ (50 steps; green), $\beta = 0.5$ (2 steps; yellow), respectively.

<img src="../img/3-gradients-time-series-i1.png" width="250" height="150">
<img src="../img/3-gradients-time-series-i2.png" width="250" height="150">
<img src="../img/3-gradients-time-series-i3.png" width="250" height="150">

So, the closer $\beta$ is to 1, the smoother is the exponentially weighted average, as it considers more previous steps, adapting more slowly to oscillations.

<u>Gradient descent with momentum</u> attempts to be resistant to gradient oscillations.

Recall we used $\nabla_{\underline w} C$ to denote the gradient vector of the parameter vector $\underline w$ with respect to the cost $C$.
$$
\begin{align*}
v_t &= \beta v_{t-1} + (1-\beta) \nabla_{\underline w}C\\
w_t &= w_{t-1} + \eta v_t
\end{align*}
$$

As we can see, it changes weights [not in the direction of the current gradient] but [in the direction of the exponentially weighted average of the current and previous gradients]. These averages oscillate less than individual gradients. As a result:
- convergence is faster because of accounting less for gradient oscillations;
- convergence is faster also because we can now use a larger learning rate.
- can prevent going towards local minima. Experimentally, it will also power through plateau regions better.


# RMSProp (root mean squares prop)
<img src="../img/5-gradients-rmsprop.png" width="850">

RMSProp also attempts to be resistant to gradient oscillations. It maintains an exponentially weighted average of the (entry-wise) squared gradients in a window of previous time steps. It then divides the current gradient vector with that average, entry-wise.

Think about the weight space. Consider one weight dimension with large oscillations in the direction of derivatives, as time progresses. The moving average of squared derivatives of that weight will be large. So, dividing the current derivative by the moving average will result in a small number. The current weight update will then be smaller. So, weight updates will be less susceptibel to gradient oscillations.

$$
\begin{align}
    s_t &= \beta s_{t-1} + (1 - \beta) \overline{\underline w} \otimes \overline{\underline w}\\
    \underline w &= \underline w - \eta \overline{\underline w} (/) (\sqrt{\underline s_t} + \epsilon)
\end{align}
$$
where $(/)$ is an attempt to denote element-wise division.

Consider the image above.
In the second equation:
- we update $w_1$ as $w_1 = w_1 - \eta \overline w_1 / (\sqrt{s_{t,1}} + \epsilon)$. But $s_{t,1}$ (corresponding to $w_1$, the horizontal axis) will be small, so progress in the direction of $w_1$ will be large.
- we update $w_2$ as $w_2 = w_2 - \eta \overline w_2 / (\sqrt{s_{t,1}} + \epsilon)$. But $s_{t,2}$, (corresponding to $w_2$, the vertical axis) will be large, so progress in the direction of $w_2$ will be small.

Advantages:
- Same as Momentum;
- Squared gradients penalises changing a specific weight dimension over and over again. It gives a chance to other dimensions to catch up. This makes it good for <u>saddle points</u>, when one one dimension does not change much, but the other does. If the second dimensions did not change much in the past, now it has the chance to change. On the other hand, the first dimension will not change much, as a result we will not navigate on the saddle point for long.

## Momentum vs RMSProp
Momentum computes an exponential weighted average of previous gradients. It then replaces the current gradient with that average in the weight-update equation of gradient descent.

RMSProp computes an exponential weighted average of previous gradients <u>squared</u>. It then divides the current gradient by that average entry wise, and uses that in the weight-update equation of gradient descent.

# Adam (Adaptive Moment Estimation)

Adam is the direct merging of Momentum and RMSProp. It also does bias correction for exponential weighted averages.

Big steps when the gradients do not change much and small steps when they vary rapidly.

\begin{align}
    m_t &= \beta_1 m_{t-1} + (1 - \beta_1) \overline{\underline w}\\
    v_t &= \beta_2 v_{t-1} + (1 - \beta_2) (\overline{\underline w} \otimes \overline{\underline w})\\
    \underline w &= \underline w - \eta m_t (/) (\sqrt{\underline s_t} + \epsilon)
\end{align}

As mt and vt are initialized as vectors of 0's, the authors of Adam observe that they are biased towards zero, especially during the initial time steps, and especially when the decay rates are small (i.e. β1 and β2 are close to 1).


vs AdamW:

Key Differences

    Weight Decay:
        In Adam, weight decay is part of the gradient update, which can distort the optimization process.
        In AdamW, weight decay is decoupled and applied independently, leading to more stable and effective training, especially in large models.

    
Unlike Adam, where weight decay is linked implicitly to the learning rate, AdamW decouples weight decay from the optimization process. This means that in AdamW, the learning rate and weight decay are optimized separately. Consequently, adjusting the learning rate does not require re-calculating the optimal weight decay, resulting in a more stable optimization.

The regularization term is added to the cost function which is then derived to calculate the gradients g. However, if one adds the weight decay term at this point, the moving averages of the gradient and its square (m and v) keep track not only of the gradients of the loss function but also of the regularization term!

As you can see the weight decay is normalized by sqrt(v) as well. If the gradient of a certain weight is large (or is changing a lot), the corresponding v is large too and the weight is regularized less than weights with small and slowly changing gradients! This means that L2 regularization does not work as intended and is not as effective as with SGD which is why SGD yields models that generalize better and has been used for most state-of-the-art results.

The authors, therefore, suggest an improved version of Adam called AdamW where the weight decay is performed only after controlling the parameter-wise step size (see the green term in line 12). The weight decay or regularization term does not end up in the moving averages and is thus only proportional to the weight itself.

# Check your derivatives!
One way to check that the derivatives computed are correct is by computing _finite differences_:
$$
\frac{\partial C(w)}{\partial w} \approx \frac{f(w + \epsilon / 2) - f(w - \epsilon / 2)}{\epsilon}
$$
for $\epsilon = 10^{-5}$.
Of course, we could not actually compute derivatives like this. We need to evaluate each dimension $d$ in turn. This is $D$ times worse then, and less accurate.

Resources:
- https://www.ruder.io/optimizing-gradient-descent/#gradientdescentoptimizationalgorithms